In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import train_test_split

In [4]:
train_dir="./ml-100k/u1.base"
test_dir="./ml-100k/u1.test"

In [15]:
def readData(dir):
    data=[]
    with open(dir,"r") as file:
        for line in file:
            x=line
            tuple=[int(x.split()[0]),int(x.split()[1]),int(x.split()[2])]
            data.append(tuple)            
        return np.array(data)
        

In [17]:
train_data=readData(train_dir)
test_data=readData(test_dir)